Importing the necessary libraries

In [1]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

from geopy.geocoders import Nominatim


from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

!pip install folium
import folium

    100% |████████████████████████████████| 92kB 7.8MB/s eta 0:00:01
Requirement not upgraded as not directly required: numpy in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: requests in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: jinja2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: six in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from folium)
Requirement not upgraded as not directly required: chardet<3.1.0,>=3.0.2 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: idna<2.7,>=2.5 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from requests->folium)
Requirement not upgraded as not directly required: urllib3<1.23,>=1.21.1 in /opt/conda/envs/DSX-Py

Defining the url of the Wikipedia page and groom for data scraping

In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
page = requests.get(url)
doc = lh.fromstring(page.content)
tr_elements = doc.xpath('//tr')

Filter the different items from the table and add them to a dictionary to create the table

In [3]:
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [4]:
tr_elements = doc.xpath('//tr')

col=[]
i=0
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    col.append((name,[]))

In [5]:
for j in range(1,len(tr_elements)):
    T=tr_elements[j]
    
    if len(T)!=3:
        break
    
    i=0
    
    for t in T.iterchildren():
        data=t.text_content() 
        if i>0:
            try:
                data=int(data)
            except:
                pass
        col[i][1].append(data)
        i+=1

In [6]:
[len(C) for (title,C) in col]

[289, 289, 289]

In [7]:
Dict={title:column for (title,column) in col}
df = pd.DataFrame.from_dict(Dict)

Drop rows that have "Not assigned" in the Borough column

In [8]:
df = df[df.Borough != "Not assigned"]

Rename Neighbourhood column

In [9]:
df.rename(index=str, columns={"Neighbourhood\n": "Neighbourhood"}, inplace=True)

Make the "Not assigned" values in the Neighbourhood column equal to the Borough column

In [10]:
df.Neighbourhood = df.Borough.where(df.Neighbourhood == 'Not assigned\n', df.Neighbourhood)

Group by Postcode and Join the values in Neighbourhood

In [11]:
df = df.groupby('Postcode', as_index=False).agg({'Borough':'first', 'Neighbourhood':', '.join})

Remove unwaned \n characters

In [12]:
df = df.replace(r'\n',' ', regex=True) 

Rename Postcode column

In [13]:
df.rename(index=str, columns={"Postcode": "PostalCode"}, inplace=True)

In [14]:
df.head()

,PostalCode,Neighbourhood,Borough
0,M1B,"Rouge , Malvern",Scarborough
1,M1C,"Highland Creek , Rouge Hill , Port Union",Scarborough
2,M1E,"Guildwood , Morningside , West Hill",Scarborough
3,M1G,Woburn,Scarborough
4,M1H,Cedarbrae,Scarborough


Get dataframe shape

In [15]:
df.shape

(103, 3)

Get the Geo Data, since the method described in the assignment didn't work for me, but it said using the csv was ok.

In [16]:
location = pd.read_csv('https://cocl.us/Geospatial_data')

In [17]:
location.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Converting and Renaming Columns

In [18]:
location.rename(index=str, columns={"Postal Code": "PostalCode"}, inplace=True)

In [19]:
location['Latitude'] = location['Latitude'].astype(float)
location['Longitude'] = location['Longitude'].astype(float)

Merge the Dataframes

In [20]:
df_complete = pd.merge(df,
                 location[['PostalCode', 'Latitude', 'Longitude']],
                 on='PostalCode')

In [21]:
df_complete = df_complete[df_complete['Borough'].str.contains("Toronto")]

In [22]:
df_complete.head()

,PostalCode,Neighbourhood,Borough,Latitude,Longitude
37,M4E,The Beaches,East Toronto,43.676357,-79.293031
41,M4K,"The Danforth West , Riverdale",East Toronto,43.679557,-79.352188
42,M4L,"The Beaches West , India Bazaar",East Toronto,43.668999,-79.315572
43,M4M,Studio District,East Toronto,43.659526,-79.340923
44,M4N,Lawrence Park,Central Toronto,43.728020,-79.388790


Defining data for geographical map plotting

In [23]:
address = 'Toronto'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [24]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, label in zip(df_complete['Latitude'], df_complete['Longitude'], df_complete['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,).add_to(map_toronto)  
    
map_toronto

Importing my User Credentials

In [25]:
CLIENT_ID = '12YFPIWA3PDBL22NCCOYRGFZ4IFSQ0IHWJ3VDEPOMNRRDWIR' 
CLIENT_SECRET = 'AMMWYSSUBX0QQUASNPX2ZBWYL1SE0OSN5KQWLGBYDKDC2I3K'
VERSION = '20180605'
LIMIT = 100
radius = 500 

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 12YFPIWA3PDBL22NCCOYRGFZ4IFSQ0IHWJ3VDEPOMNRRDWIR
CLIENT_SECRET:AMMWYSSUBX0QQUASNPX2ZBWYL1SE0OSN5KQWLGBYDKDC2I3K


Get a list of nearby venues via API

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        results = requests.get(url).json()["response"]['groups'][0]['items']
    
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get the defined nearby venues list for Toronto neighbourhoods

In [27]:
toronto_venues = getNearbyVenues(names=df_complete['Neighbourhood'],
                                   latitudes=df_complete['Latitude'],
                                   longitudes=df_complete['Longitude']
                                  )

The Beaches 
The Danforth West , Riverdale 
The Beaches West , India Bazaar 
Studio District 
Lawrence Park 
Davisville North 
North Toronto West 
Davisville 
Moore Park , Summerhill East 
Deer Park , Forest Hill SE , Rathnelly , South Hill , Summerhill West 
Rosedale 
Cabbagetown , St. James Town 
Church and Wellesley 
Harbourfront , Regent Park 
Ryerson , Garden District 
St. James Town 
Berczy Park 
Central Bay Street 
Adelaide , King , Richmond 
Harbourfront East , Toronto Islands , Union Station 
Design Exchange , Toronto Dominion Centre 
Commerce Court , Victoria Hotel 
Roselawn 
Forest Hill North , Forest Hill West 
The Annex , North Midtown , Yorkville 
Harbord , University of Toronto 
Chinatown , Grange Park , Kensington Market 
CN Tower , Bathurst Quay , Island airport , Harbourfront West , King and Spadina , Railway Lands , South Niagara 
Stn A PO Boxes 25 The Esplanade 
First Canadian Place , Underground city 
Christie 
Dovercourt Village , Dufferin 
Little Portugal , Trini

In [28]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 235 unique categories.


Onehot Encoding for Data, adding the Neighborhoods to the Dataframe

In [29]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Piano Bar,Pizza Place,Plane,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

Group by Neighbourhood

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape

(38, 235)

Get top 5 Venues per Neighborhood

In [31]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide , King , Richmond ----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2  American Restaurant  0.04
3      Thai Restaurant  0.04
4           Steakhouse  0.04


----Berczy Park ----
            venue  freq
0     Coffee Shop  0.07
1    Cocktail Bar  0.05
2      Restaurant  0.05
3             Pub  0.04
4  Farmers Market  0.04


----Brockton , Exhibition Place , Parkdale Village ----
               venue  freq
0               Café  0.11
1     Breakfast Spot  0.11
2        Coffee Shop  0.11
3  Convenience Store  0.05
4      Grocery Store  0.05


----Business Reply Mail Processing Centre 969 Eastern ----
                venue  freq
0  Light Rail Station  0.11
1       Burrito Place  0.05
2       Garden Center  0.05
3              Garden  0.05
4                 Spa  0.05


----CN Tower , Bathurst Quay , Island airport , Harbourfront West , King and Spadina , Railway Lands , South Niagara ----
              venue  freq
0    Airport Lounge  0.1

Create new Dataframe with top venues

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide , King , Richmond",Coffee Shop,Café,Steakhouse,American Restaurant,Thai Restaurant,Gym,Bakery,Clothing Store,Asian Restaurant,Bar
1,Berczy Park,Coffee Shop,Cocktail Bar,Restaurant,Pub,Café,Cheese Shop,Seafood Restaurant,Farmers Market,Italian Restaurant,Steakhouse
2,"Brockton , Exhibition Place , Parkdale Village",Coffee Shop,Café,Breakfast Spot,Gym,Furniture / Home Store,Pet Store,Nightclub,Climbing Gym,Caribbean Restaurant,Burrito Place
3,Business Reply Mail Processing Centre 969 East...,Light Rail Station,Yoga Studio,Auto Workshop,Comic Shop,Pizza Place,Butcher,Recording Studio,Restaurant,Burrito Place,Brewery
4,"CN Tower , Bathurst Quay , Island airport , Ha...",Airport Lounge,Airport Terminal,Airport Service,Harbor / Marina,Boat or Ferry,Sculpture Garden,Plane,Boutique,Airport Gate,Airport


Cluster the Dataframe in 5 clusters with k-means

In [33]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

Add Labels and Merge Dataframes

In [34]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_complete

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')


In [35]:
toronto_merged.shape

(38, 16)

Drop columns with "NaN"

In [36]:
toronto_merged = toronto_merged.dropna()

Convert Labels columns to integer, since otherwise the map won't plot

In [37]:
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)


Create the map

In [38]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Show final table

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
37,The Beaches,0,Coffee Shop,Pub,Women's Store,Diner,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant
41,"The Danforth West , Riverdale",0,Greek Restaurant,Coffee Shop,Ice Cream Shop,Bookstore,Italian Restaurant,Health Food Store,Furniture / Home Store,Pub,Pizza Place,Liquor Store
42,"The Beaches West , India Bazaar",0,Fast Food Restaurant,Sandwich Place,Italian Restaurant,Sushi Restaurant,Burrito Place,Pub,Burger Joint,Ice Cream Shop,Fish & Chips Shop,Movie Theater
43,Studio District,0,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Chinese Restaurant,Fish Market,Juice Bar,Latin American Restaurant,Seafood Restaurant
45,Davisville North,0,Clothing Store,Gym,Sandwich Place,Burger Joint,Park,Breakfast Spot,Hotel,Food & Drink Shop,Dance Studio,Dumpling Restaurant
46,North Toronto West,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Fast Food Restaurant,Mexican Restaurant,Diner,Dessert Shop,Park,Chinese Restaurant,Rental Car Location
47,Davisville,0,Sandwich Place,Dessert Shop,Pizza Place,Sushi Restaurant,Coffee Shop,Restaurant,Café,Italian Restaurant,Seafood Restaurant,Toy / Game Store
49,"Deer Park , Forest Hill SE , Rathnelly , South...",0,Coffee Shop,Pub,Fried Chicken Joint,Light Rail Station,Sushi Restaurant,Bagel Shop,Sports Bar,American Restaurant,Pizza Place,Convenience Store
51,"Cabbagetown , St. James Town",0,Coffee Shop,Restaurant,Park,Pub,Italian Restaurant,Bakery,Pizza Place,Café,Breakfast Spot,Caribbean Restaurant
52,Church and Wellesley,0,Japanese Restaurant,Coffee Shop,Sushi Restaurant,Gay Bar,Restaurant,Burger Joint,Men's Store,Gastropub,Mediterranean Restaurant,Pub
